## Tree building evaluation on gold EDUs (mostly) and playground for tree building scripts

1. Modifications of library components for tree building
2. Scripts for test and evaluation of Sklearn-, AllenNLP- and gold-annotation-based RST parsers on manually segmented corpus

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from utils.print_tree import printBTree
#from utils.rst_annotation import DiscourseUnit

import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')

In [3]:
from isanlp.annotation_rst import DiscourseUnit

In [4]:
def printTree(tree):
    def _(n):
        if n.relation:
            value = (n.relation, "%.2f"%(n.proba))
        else:
            value = n.text
        return str(value), n.left, n.right

    return printBTree(_) 

In [5]:
class DiscourseUnitCreator:
    def __init__(self, id):
        self.id = id
        
    def __call__(self, left_node, right_node, proba):
        self.id += 1
        return DiscourseUnit(
            id=id,
            left=left_node,
            right=right_node,
            relation=1,
            proba=proba
        )

In [7]:
from allennlp.predictors import Predictor

pr = Predictor.from_path('../isanlp_rst/models/structure_predictor_lstm/model.tar.gz')

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [129]:
from isanlp_rst.src.isanlp_rst.sklearn_classifier import SklearnClassifier
from isanlp_rst.src.isanlp_rst.allennlp_classifier import AllenNLPClassifier
from isanlp_rst.src.isanlp_rst.rst_tree_predictor import *
from isanlp_rst.src.isanlp_rst.greedy_rst_parser import GreedyRSTParser
from isanlp_rst.src.isanlp_rst.features_extractor import FeaturesExtractor
from isanlp_rst.src.isanlp_rst.features_processor_tokenizer import FeaturesProcessor

In [12]:
from utils.train_test_split import split_train_dev_test

train, dev, test = split_train_dev_test('./data')

news in train: 0.5344827586206896,	in dev: 0.6470588235294118,	in test: 0.6086956521739131
ling in train: 0.0,	in dev: 0.0,	in test: 0.0
comp in train: 0.0,	in dev: 0.0,	in test: 0.0
blog in train: 0.43103448275862066,	in dev: 0.5294117647058824,	in test: 0.4782608695652174


# Evaluation (Parser)

In [16]:
_SPAN_PREDICTOR = {
    'lstm': (AllenNLPClassifier, 'structure_predictor_lstm', 0.1, 0.5),
    'ensemble': (SklearnClassifier, 'structure_predictor', 0.15, 0.2),
}

_LABEL_PREDICTOR = {
    'lstm': (AllenNLPClassifier, 'label_predictor_lstm'),
    'ensemble': (SklearnClassifier, 'label_predictor'),
}

In [17]:
binary_classifier = AllenNLPClassifier('../isanlp_rst/models/structure_predictor_lstm/')
label_classifier = AllenNLPClassifier('../isanlp_rst/models/label_predictor_lstm/')

features_processor = FeaturesProcessor(model_dir_path='models', verbose=False)
features_extractor = FeaturesExtractor(features_processor)

predictor = NNTreePredictor(features_processor=features_extractor, 
                            relation_predictor_sentence=None,
                            relation_predictor_text=binary_classifier, 
                            label_predictor=label_classifier)

paragraph_parser = GreedyRSTParser(predictor,
                                   confidence_threshold=_SPAN_PREDICTOR['lstm'][2])

document_parser = GreedyRSTParser(predictor,
                                  confidence_threshold=_SPAN_PREDICTOR['lstm'][3])

In [683]:
additional_document_parser = GreedyRSTParser(predictor,
                                             confidence_threshold=_SPAN_PREDICTOR['lstm'][3]-0.15)

In [137]:
from isanlp.annotation import Sentence

def split_by_paragraphs(annot_text, annot_tokens, annot_sentences, annot_lemma, annot_morph, annot_postag,
                        annot_syntax_dep_tree):

    def split_on_two(sents, boundary):
        list_sum = lambda l: sum([len(sublist) for sublist in l])

        i = 1
        while list_sum(sents[:i]) < boundary and i < len(sents):
            i += 1

        intersentence_boundary = min(len(sents[i - 1]), boundary - list_sum(sents[:i - 1]))
        return (sents[:i - 1] + [sents[i - 1][:intersentence_boundary]],
                [sents[i - 1][intersentence_boundary:]] + sents[i:])

    def recount_sentences(chunk):
        sentences = []
        lemma = []
        morph = []
        postag = []
        syntax_dep_tree = []
        tokens_cursor = 0

        for i, sent in enumerate(chunk['syntax_dep_tree']):
            if len(sent) > 0:
                sentences.append(Sentence(tokens_cursor, tokens_cursor + len(sent)))
                lemma.append(chunk['lemma'][i])
                morph.append(chunk['morph'][i])
                postag.append(chunk['postag'][i])
                syntax_dep_tree.append(chunk['syntax_dep_tree'][i])
                tokens_cursor += len(sent)

        chunk['sentences'] = sentences
        chunk['lemma'] = lemma
        chunk['morph'] = morph
        chunk['postag'] = postag
        chunk['syntax_dep_tree'] = syntax_dep_tree

        return chunk

    chunks = []
    prev_right_boundary = -1

    for i, token in enumerate(annot_tokens[:-1]):

        if '\n' in annot_text[token.end:annot_tokens[i + 1].begin]:
            if prev_right_boundary > -1:
                chunk = {
                    'text': annot_text[annot_tokens[prev_right_boundary].end:token.end + 1].strip(),
                    'tokens': annot_tokens[prev_right_boundary + 1:i + 1]
                }
            else:
                chunk = {
                    'text': annot_text[:token.end + 1].strip(),
                    'tokens': annot_tokens[:i + 1]
                }

            lemma, annot_lemma = split_on_two(annot_lemma, i - prev_right_boundary)
            morph, annot_morph = split_on_two(annot_morph, i - prev_right_boundary)
            postag, annot_postag = split_on_two(annot_postag, i - prev_right_boundary)
            syntax_dep_tree, annot_syntax_dep_tree = split_on_two(annot_syntax_dep_tree, i - prev_right_boundary)

            chunk.update({
                'lemma': lemma,
                'morph': morph,
                'postag': postag,
                'syntax_dep_tree': syntax_dep_tree,
            })
            chunks.append(recount_sentences(chunk))

            prev_right_boundary = i  # number of last token in the last chunk

    chunk = {
        'text': annot_text[annot_tokens[prev_right_boundary].end:].strip(),
        'tokens': annot_tokens[prev_right_boundary + 1:],
        'lemma': annot_lemma,
        'morph': annot_morph,
        'postag': annot_postag,
        'syntax_dep_tree': annot_syntax_dep_tree,
    }

    chunks.append(recount_sentences(chunk))
    return chunks

In [738]:
cache = {}

In [566]:
annot['text']

'Новость: Выставка песчаной скульптуры "Волшебный песок"\nУникальная выставка песчаной скульптуры "Волшебный песок" действует на пл.Речников (Речной порт). Для создания скульптур из капризного материала, как песок приехали скульпторы из Архангельска, Санкт-Петербурга, Ижевска, Екатеринбурга, Нижнего Тагила, Латвии,г.Лиепая, так же участвовали скульпторы из Чебоксары. Скульпторы из российских городов, Ижевска, Нижнего Тагила, Санкт-Петербурга, Архангельска, Чебоксар Латвии соорудили настоящие шедевры из песка. Конкуренцию им составили двое местных мастеров, один из которых - организатор выставки Андрей Молоков. И это было только начало. Через несколько дней отдельный конкурс был организован для чебоксарских студентов художественных факультетов и училищ.\nТема выставки - «Песчаные замки». По условиям конкурса в композиции обязательно должны присутствовать элементы архитектуры. Это может быть что угодно: дома, замки, крепости - всё, что подскажет фантазия автора.\nПесчаный город состоит и

In [610]:
def split_by_paragraphs_edus(edus, text):
    res = []
    parag = []
    
    for edu in edus:
        parag.append(edu)
        boundary = text.find(edu)+len(edu)
        if boundary < len(text):
            if text[boundary] == '\n':
                res.append(parag)
                parag = []
         
    if parag:
        res.append(parag)
    return res

In [611]:
split_by_paragraphs_edus(edus, annot['text'])

[['Новость: Выставка песчаной скульптуры "Волшебный песок"'],
 ['Уникальная выставка песчаной скульптуры "Волшебный песок" действует на пл.Речников (Речной порт).',
  'Для создания скульптур из капризного материала, как песок',
  'приехали скульпторы из Архангельска, Санкт-Петербурга, Ижевска, Екатеринбурга, Нижнего Тагила, Латвии,г.Лиепая,',
  'так же участвовали скульпторы из Чебоксары.',
  'Скульпторы из российских городов, Ижевска, Нижнего Тагила, Санкт-Петербурга, Архангельска, Чебоксар Латвии соорудили настоящие шедевры из песка.',
  'Конкуренцию им составили двое местных мастеров,',
  'один из которых - организатор выставки Андрей Молоков.',
  'И это было только начало.',
  'Через несколько дней отдельный конкурс был организован для чебоксарских студентов художественных факультетов и училищ.'],
 ['Тема выставки - «Песчаные замки».',
  'По условиям конкурса в композиции обязательно должны присутствовать элементы архитектуры.',
  'Это может быть что угодно: дома, замки, крепости',

In [637]:
cache = []

In [659]:
def prepare_gold_pairs(gold_pairs):
    TARGET = 'category_id'

    gold_pairs[TARGET] = gold_pairs[TARGET].replace([0.0], 'same-unit_m')
    gold_pairs['order'] = gold_pairs['order'].replace([0.0], 'NN')
    gold_pairs[TARGET] = gold_pairs[TARGET].replace(['antithesis_r',], 'contrast_m')
    gold_pairs[TARGET] = gold_pairs[TARGET].replace(['cause_r', 'effect_r'], 'cause-effect_r')
    gold_pairs[TARGET] = gold_pairs[TARGET].replace(['conclusion_r',], 'restatement_m')
    gold_pairs[TARGET] = gold_pairs[TARGET].replace(['evaluation_r'], 'interpretation-evaluation_r')
    gold_pairs[TARGET] = gold_pairs[TARGET].replace(['motivation_r',], 'condition_r')
    gold_pairs['relation'] = gold_pairs[TARGET].map(lambda row: row[:-1]) + gold_pairs['order']
    gold_pairs['relation'].value_counts()
    gold_pairs['relation'] = gold_pairs['relation'].replace(['restatement_SN', 'restatement_NS'], 'restatement_NN')
    gold_pairs['relation'] = gold_pairs['relation'].replace(['contrast_SN', 'contrast_NS'], 'contrast_NN')
    gold_pairs['relation'] = gold_pairs['relation'].replace(['solutionhood_NS', 'preparation_NS'], 'elaboration_NS')
    gold_pairs['relation'] = gold_pairs['relation'].replace(['concession_SN', 'evaluation_SN', 
                                                             'elaboration_SN', 'evidence_SN'], 'preparation_SN')

    _class_mapper = {
            'background_NS': 'elaboration_NS',
            'background_SN': 'preparation_SN',
            'comparison_NN': 'contrast_NN',
            'interpretation-evaluation_SN': 'elaboration_NS',
            'evidence_NS': 'elaboration_NS',
            'restatement_NN': 'joint_NN',
            'sequence_NN': 'joint_NN'
        }

    for key, value in _class_mapper.items():
        gold_pairs['relation'] = gold_pairs['relation'].replace(key, value)
        
    gold_pairs['order'] = gold_pairs['relation'].map(lambda row: row.split('_')[1])
    gold_pairs[TARGET] = gold_pairs['relation'].map(lambda row: row.split('_')[0])
        
    return gold_pairs

In [710]:
annot['text']

'https://odin-moу-den.livejournal.com/2215985.html\nВсем здравствуйте!\nМеня зовут Мария, я живу в Москве, и это один мой рабочий день.\nВообще-то я собиралась показать вам вчерашний рабочий день, по средам и воскресеньям моя дорога на работу гораздо красочнее, и внутри места, где эта работа проходит, красивее, и движухи больше. Но в среду я напрочь забыла о своем намерении снимать ОМД. Так что показываю место работы, где я бываю по вторникам и четвергам.\nКак вы уже поняли, будет много текста, и 53 телефонофотографии.\nГде-то в сообществе есть еще два моих поста, но я не смогла их найти.\nIMG\nИтак, 8-55, меня разбудил будильник. Так как легла я оочень для себя поздно, будильник я сегодня ненавижу. Долго крутилась в постели, пытаясь заставить телефон что-нибудь "увидеть". В зимних сумерках "увидеть" он смог только окно. Вот и оно. На подоконнике махровый плед - котиковое хозяйство. Самого кота там нет. Он в изгнании на кухне.\nIMG\nНа окнах сетки "антикошка". Из-за этого я вечно чувст

In [735]:
from tqdm import tqdm_notebook as tqdm
from utils.file_reading import *
from utils.evaluation import extr_pairs, extr_pairs_forest


broken_files = []
smallest_file = 'data/news2_4.edus'
coolest_file = 'data/blogs_17.edus'
#test[:1]
for file in tqdm(test):
    filename = '.'.join(file.split('.')[:-1])
    edus = read_edus(filename)
    #gold = read_gold(filename)
    gold = prepare_gold_pairs(read_gold(filename, features=True))
    
    annot = read_annotation(filename)
    annot['text'] = annot['text'].replace('\nIMG', ' IMG')
    
    if '\n' in annot['text']:
        chunks = split_by_paragraphs(
            annot['text'],
            annot['tokens'], 
            annot['sentences'], 
            annot['lemma'], 
            annot['morph'], 
            annot['postag'], 
            annot['syntax_dep_tree'])
        
        chunked_edus = split_by_paragraphs_edus(edus, annot['text'])
    
    dus = []
    for i, chunk in enumerate(chunks):
        _edus = []
        last_end = 0
        
        for max_id in range(len(chunked_edus[i])):
            start = len(annot['text'][:last_end]) + annot['text'][last_end:].find(chunked_edus[i][max_id])
            end = start + len(chunked_edus[i][max_id])
            temp = DiscourseUnit(
                    id=max_id,
                    left=None,
                    right=None,
                    relation='edu',
                    start=start,
                    end=end,
                    orig_text=annot['text'],
                    proba=1.,
                )

            _edus.append(temp)
            last_end = end + 1
            
        if len(_edus) == 1:
            dus += _edus
            start_id = _edus[-1].id + 1

        elif len(_edus) > 1:
            trees = paragraph_parser(_edus,
                annot['text'], chunk['tokens'], chunk['sentences'], chunk['lemma'],
                chunk['morph'], chunk['postag'], chunk['syntax_dep_tree'])
            
            dus += trees
#             print('::: chunk processed :::')
#             print(dus[-1].text)
            start_id = dus[-1].id + 1
        
    parsed = document_parser(
                dus, 
                annot['text'], 
                annot['tokens'], 
                annot['sentences'], 
                annot['lemma'], 
                annot['morph'], 
                annot['postag'], 
                annot['syntax_dep_tree'],
                genre=filename.split('_')[0])
    
    if len(parsed) > len(annot['text']) // 400:
        parsed = additional_document_parser(
            parsed, 
            annot['text'], 
            annot['tokens'], 
            annot['sentences'], 
            annot['lemma'], 
            annot['morph'], 
            annot['postag'], 
            annot['syntax_dep_tree'],
            genre=filename.split('_')[0]
        )
    
    parsed_pairs = pd.DataFrame(extr_pairs_forest(parsed, annot['text']), 
                                columns=['snippet_x', 'snippet_y', 'category_id', 'order'])
    evaluation = eval_pipeline(parsed, edus, gold, annot['text'])
    evaluation['filename'] = file
    cache.append(evaluation)

/notebook/isanlp_rst/src/isanlp_rst/features_processor_tokenizer.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['loc_x'] = df.snippet_x.map(self.annot_text.find)
/notebook/isanlp_rst/src/isanlp_rst/features_processor_tokenizer.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['loc_y'] = df.apply(lambda row: self._find_y(row.snippet_x, row.snippet_y, row.loc_x), axis=1)
/notebook/isanlp_rst/src/isanlp_rst/features_processor_tokenizer.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [737]:
filename

'./data/blogs_31'

In [725]:
edus

['https://odin-moу-den.livejournal.com/2215985.html',
 'Всем здравствуйте!',
 'Меня зовут Мария,',
 'я живу в Москве,',
 'и это один мой рабочий день.',
 'Вообще-то я собиралась показать вам вчерашний рабочий день,',
 'по средам и воскресеньям моя дорога на работу гораздо красочнее,',
 'и внутри места, где эта работа проходит, красивее,',
 'и движухи больше.',
 'Но в среду я напрочь забыла о своем намерении снимать ОМД.',
 'Так что показываю место работы,',
 'где я бываю по вторникам и четвергам.',
 'Как вы уже поняли, будет много текста, и 53 телефонофотографии.',
 'Где-то в сообществе есть еще два моих поста,',
 'но я не смогла их найти. IMG',
 'Итак, 8-55, меня разбудил будильник.',
 'Так как легла я оочень для себя поздно,',
 'будильник я сегодня ненавижу.',
 'Долго крутилась в постели,',
 'пытаясь заставить телефон что-нибудь "увидеть".',
 'В зимних сумерках "увидеть" он смог только окно. Вот и оно.',
 'На подоконнике махровый плед - котиковое хозяйство.',
 'Самого кота там нет.',

In [728]:
[tok.text for tok in chunks[5]['tokens']]

['Где',
 '-',
 'то',
 'в',
 'сообществе',
 'есть',
 'еще',
 'два',
 'моих',
 'поста',
 ',',
 'но',
 'я',
 'не',
 'смогла',
 'их',
 'найти',
 '.']

In [727]:
chunked_edus[5]

['Где-то в сообществе есть еще два моих поста,',
 'но я не смогла их найти. IMG',
 'Итак, 8-55, меня разбудил будильник.',
 'Так как легла я оочень для себя поздно,',
 'будильник я сегодня ненавижу.',
 'Долго крутилась в постели,',
 'пытаясь заставить телефон что-нибудь "увидеть".',
 'В зимних сумерках "увидеть" он смог только окно. Вот и оно.',
 'На подоконнике махровый плед - котиковое хозяйство.',
 'Самого кота там нет.',
 'Он в изгнании на кухне. IMG',
 'На окнах сетки "антикошка".',
 'Из-за этого я вечно чувствую себя в заточении,',
 'даже в окно выглядывать не охота. IMG',
 'Моя сонная физиономия, грязное зеркало и микрованная (моя боль и печаль).',
 'Угу, к концу года я устала,',
 'поэтому слегка в миноре. IMG',
 'Иду в комнату включить комп',
 'и впустить кота.',
 'Нет, ВПУСТИТЬ кота.',
 'Он каждый раз так влетает,',
 'будно всю ночь только об этом и мечтал.']

In [711]:
for edu in _edus:
    print(edu.text)
    print(annot['text'].find(edu.text))

Нет, это не бардак на столе.
1313
Бардак на столе был пару недель назад.
1342
А сейчас - это так, легкий хаосик: бумаги с работы, мой предновогодний подарок, мобильный и разного по мелочи.
1381
Знаете, бывают подарки новогодние,
1492
бывают - предновогодние.
1527
Первые иногда становятся вторыми,
1552
если даритель или одариваемый недотерпел,
1586
а бывают специально предновогодними.
1628
Вот этот учебник - он как раз такой,
1665
я купила его себе в подарок,
1702
для приятности.
1731


In [705]:
filename

'./data/blogs_21'

In [706]:
for tree in dus:
    print('>>>', tree.relation + '_' + tree.nuclearity)
    if tree.left:
        print(tree.text)

>>> edu_
>>> edu_
>>> joint_NN
Меня зовут Мария, я живу в Москве, и это один мой рабочий день.
>>> contrast_NN
Вообще-то я собиралась показать вам вчерашний рабочий день, по средам и воскресеньям моя дорога на работу гораздо красочнее, и внутри места, где эта работа проходит, красивее, и движухи больше. Но в среду я напрочь забыла о своем намерении снимать ОМД. Так что показываю место работы, где я бываю по вторникам и четвергам.
>>> edu_
>>> elaboration_NS
Где-то в сообществе есть еще два моих поста, но я не смогла их найти.
IMG
Итак, 8-55, меня разбудил будильник. Так как легла я оочень для себя поздно, будильник я сегодня ненавижу. Долго крутилась в постели, пытаясь заставить телефон что-нибудь "увидеть". В зимних сумерках "увидеть" он смог только окно. Вот и оно. На подоконнике махровый плед - котиковое хозяйство. Самого кота там нет. Он в изгнании на кухне.
IMG
На окнах сетки "антикошка". Из-за этого я вечно чувствую себя в заточении, даже в окно выглядывать не охота.
IMG
Моя сонн

In [730]:
from utils.evaluation import eval_pipeline

evaluation = eval_pipeline(parsed, edus, gold, annot['text'])
evaluation['filename'] = file
cache = [evaluation]

In [731]:
test[0]

'./data/blogs_17.edus'

In [736]:
tmp = pd.DataFrame(cache)
tmp['pr_seg'] = tmp.seg_true_pred / tmp.seg_all_pred
tmp['re_seg'] = tmp.seg_true_pred / tmp.seg_all_true
tmp['f1_seg'] = 2 * tmp.pr_seg * tmp.re_seg / (tmp.pr_seg + tmp.re_seg)
tmp['pr_unlab'] = tmp.unlab_true_pred / tmp.unlab_all_pred
tmp['re_unlab'] = tmp.unlab_true_pred / tmp.unlab_all_true
tmp['f1_unlab'] = 2 * tmp.pr_unlab * tmp.re_unlab / (tmp.pr_unlab + tmp.re_unlab)
tmp['pr_lab'] = tmp.lab_true_pred / tmp.lab_all_pred
tmp['re_lab'] = tmp.lab_true_pred / tmp.lab_all_true
tmp['f1_lab'] = 2 * tmp.pr_lab * tmp.re_lab / (tmp.pr_lab + tmp.re_lab)
tmp['pr_nuc'] = tmp.nuc_true_pred / tmp.nuc_all_pred
tmp['re_nuc'] = tmp.nuc_true_pred / tmp.nuc_all_true
tmp['f1_nuc'] = 2 * tmp.pr_nuc * tmp.re_nuc / (tmp.pr_nuc + tmp.re_nuc)
tmp.sort_values('f1_seg', ascending=False)

,filename,full_all_pred,full_all_true,full_true_pred,lab_all_pred,lab_all_true,lab_true_pred,nuc_all_pred,nuc_all_true,nuc_true_pred,...,f1_seg,pr_unlab,re_unlab,f1_unlab,pr_lab,re_lab,f1_lab,pr_nuc,re_nuc,f1_nuc
0,./data/blogs_21.edus,280,251,71,280,251,76,280,251,89,...,1.0,0.496429,0.553785,0.523540,0.271429,0.302789,0.286252,0.317857,0.354582,0.335217
1,./data/blogs_17.edus,86,83,25,86,83,26,86,83,34,...,1.0,0.546512,0.566265,0.556213,0.302326,0.313253,0.307692,0.395349,0.409639,0.402367
2,./data/blogs_21.edus,280,251,71,280,251,76,280,251,89,...,1.0,0.496429,0.553785,0.523540,0.271429,0.302789,0.286252,0.317857,0.354582,0.335217


,filename,full_all_pred,full_all_true,full_true_pred,lab_all_pred,lab_all_true,lab_true_pred,nuc_all_pred,nuc_all_true,nuc_true_pred,...,f1_seg,pr_unlab,re_unlab,f1_unlab,pr_lab,re_lab,f1_lab,pr_nuc,re_nuc,f1_nuc
0,data/blogs_17.edus,86,83,25,86,83,26,86,83,34,...,1.0,0.546512,0.566265,0.556213,0.302326,0.313253,0.307692,0.395349,0.409639,0.402367


In [641]:
from utils.evaluation import _not_parsed_as_in_gold

err = _not_parsed_as_in_gold(parsed_pairs, gold, labeled=True)

In [642]:
err

,snippet_x,category_id_gold,snippet_y,loc_x,loc_y,order_gold,filename,category_id_parsed,order_parsed
10,"«Моя идея заключается в том, что всё в руках Г...",cause,потому и название такое - «Невидимая сила»...»,1523,1608,NS,news2_4,concession,NS
14,"«Композиция соответствует своему названию, име...",evaluation,выполнена мастерски - очень много сложных дета...,1749,1833,NS,news2_4,joint,NN
17,"чтобы полюбоваться песчаными шедеврами и,",joint,"может быть, вдохновиться на создание своего ма...",2441,2483,NN,news2_4,purpose,SN
29,Бронзовым призёром стал Айнарас Зигнис из латв...,preparation,В своей песчаной фантазии Айнарас воплотил сво...,1919,2045,SN,news2_4,elaboration,NS


In [40]:
from utils.evaluation import metric_parseval

filenames = []
true_pos = []
all_parsed = []
all_gold = []

for key, value in cache.items():
    c_true_pos, c_all_parsed, c_all_gold = metric_parseval(value[0], value[1])
    filenames.append(key)
    true_pos.append(c_true_pos)
    all_parsed.append(c_all_parsed)
    all_gold.append(c_all_gold)
    
results = pd.DataFrame({'filename': filenames, 
                    'true_pos': true_pos,
                    'all_parsed': all_parsed,
                    'all_gold': all_gold})

In [41]:
results['recall'] = results['true_pos'] / results['all_gold']
results['precision'] = results['true_pos'] / results['all_parsed']
results['F1'] = 2 * results['precision'] * results['recall'] / (results['precision'] + results['recall'])

In [42]:
results

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news1_47,0,2,96,0.0,0.0,NaN


In [166]:
results

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news2_44,112,174,153,0.732026,0.643678,0.685015
1,data/news1_47,49,106,96,0.510417,0.462264,0.485149


In [156]:
results

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news2_44,112,174,153,0.732026,0.643678,0.685015


In [134]:
results

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news2_44,112,174,153,0.732026,0.643678,0.685015
1,data/news1_72,41,78,66,0.621212,0.525641,0.569444
2,data/news1_47,49,106,96,0.510417,0.462264,0.485149
3,data/news1_74,33,60,58,0.568966,0.550000,0.559322
4,data/news2_38,113,226,201,0.562189,0.500000,0.529274
5,data/news2_31,72,117,99,0.727273,0.615385,0.666667
6,data/news2_21,63,103,104,0.605769,0.611650,0.608696
7,data/news1_39,51,87,92,0.554348,0.586207,0.569832
8,data/news1_5,25,42,40,0.625000,0.595238,0.609756
9,data/news2_3,18,39,33,0.545455,0.461538,0.500000


In [253]:
results.to_pickle('results_20.03.pkl')

In [254]:
re = results['true_pos'].sum() / results['all_gold'].sum()
pr = results['true_pos'].sum() / results['all_parsed'].sum()
f1 = 2 * pr * re / (pr+re)
print(re, pr, f1)

0.5366718683501206 0.47388199924840285 0.503326237360298


In [258]:
tmp = results[results.filename.str.contains('ling')]
re = tmp['true_pos'].sum() / tmp['all_gold'].sum()
pr = tmp['true_pos'].sum() / tmp['all_parsed'].sum()
f1 = 2 * pr * re / (pr+re)
print(re, pr, f1)

0.5634517766497462 0.4036363636363636 0.4703389830508475


In [67]:
results.sort_values('F1')  # eee th=.3

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news2_44,106,167,153,0.69281,0.634731,0.6625


In [60]:
results.sort_values('F1')  # eee th=.2

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news2_44,112,174,153,0.732026,0.643678,0.685015


In [54]:
results.sort_values('F1')  # eee th=.15

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
0,data/news2_44,112,176,153,0.732026,0.636364,0.680851


In [209]:
results['F1'].mean(), results['recall'].mean(), results['precision'].mean()

(0.4119188649405266, 0.3704063070550079, 0.47726138642333527)

In [192]:
tmp = results[results.filename.str.contains('blog')]

tmp['F1'].mean(), tmp['recall'].mean(), tmp['precision'].mean()

(0.40254579065780033, 0.3449201654454985, 0.48762505038940707)

In [193]:
tmp = results[results.filename.str.contains('news')]

tmp['F1'].mean(), tmp['recall'].mean(), tmp['precision'].mean()

(0.4448437125215326, 0.4004284201602762, 0.5113166238620861)

In [194]:
tmp = results[results.filename.str.contains('ling')]

tmp['F1'].mean(), tmp['recall'].mean(), tmp['precision'].mean()

(0.4019318169329664, 0.37567099154362277, 0.4455345528051692)

In [195]:
tmp = results[results.filename.str.contains('comp')]

tmp['F1'].mean(), tmp['recall'].mean(), tmp['precision'].mean()

(0.3810944830999073, 0.3589145654150725, 0.41109744867219583)

# Evaluation (Gold)

In [538]:
cache = {}

In [687]:
from utils.evaluation import metric_parseval_df as metric_parseval

In [540]:
filenames = []
true_pos = []
all_parsed = []
all_gold = []

for key, value in cache.items():
    c_true_pos, c_all_parsed, c_all_gold = metric_parseval(value[0], value[1])
    filenames.append(key)
    true_pos.append(c_true_pos)
    all_parsed.append(c_all_parsed)
    all_gold.append(c_all_gold)
    
results = pd.DataFrame({'filename': filenames, 
                    'true_pos': true_pos,
                    'all_parsed': all_parsed,
                    'all_gold': all_gold})

In [541]:
from isanlp_rst.src.isanlp_rst.rst_tree_predictor import GoldTreePredictor

In [542]:
def parse_golds(filename):
    filename = '.'.join(filename.split('.')[:-1])
    edus = read_edus(filename)
    gold = read_gold(filename)
    annot = read_annotation(filename)
    
    _edus = []
    last_end = 0
    for max_id in range(len(edus)):
        start = len(annot['text'][:last_end]) + annot['text'][last_end:].find(edus[max_id])
        end = start + len(edus[max_id])
        temp = DiscourseUnit(
                id=max_id,
                left=None,
                right=None,
                relation='edu',
                start=start,
                end=end,
                orig_text=annot['text'],
                proba=1.,
            )
        _edus.append(temp)
        last_end = end

    parser = GreedyRSTParser(GoldTreePredictor(gold), confidence_threshold=0.)
    parsed = parser(_edus, annot['text'], annot['tokens'], annot['sentences'],
                    annot['postag'], annot['morph'], annot['lemma'], annot['syntax_dep_tree'])
    
    parsed_pairs = pd.DataFrame(extr_pairs_forest(parsed, annot['text']), columns=['snippet_x', 'snippet_y', 'category_id', 'order'])
    parsed_pairs[0] = parsed_pairs.snippet_x
    parsed_pairs[1] = parsed_pairs.snippet_y
    
    return (filename,) + metric_parseval(parsed_pairs, gold)#, parsed_pairs, parsed

In [550]:
%%time

import multiprocessing as mp

pool = mp.Pool(5)
result = pool.map(parse_golds, dev)
pool.close()

CPU times: user 61.1 ms, sys: 278 ms, total: 339 ms
Wall time: 9.52 s


In [551]:
results = pd.DataFrame(columns=['filename', 'true_pos', 'all_parsed', 'all_gold'], data=result)

results['recall'] = results['true_pos'] / results['all_gold']
results['precision'] = results['true_pos'] / results['all_parsed']
results['F1'] = 2 * results['precision'] * results['recall'] / (results['precision'] + results['recall'])

In [552]:
results.sort_values('F1')

,filename,true_pos,all_parsed,all_gold,recall,precision,F1
25,./data/blogs_20,53,53,151,0.350993,1.000000,0.519608
29,./data/blogs_10,43,44,83,0.518072,0.977273,0.677165
30,./data/blogs_9,72,73,134,0.537313,0.986301,0.695652
33,./data/blogs_22,195,195,361,0.540166,1.000000,0.701439
26,./data/blogs_5,64,66,105,0.609524,0.969697,0.748538
38,./data/blogs_19,167,189,255,0.654902,0.883598,0.752252
39,./data/blogs_26,41,41,67,0.611940,1.000000,0.759259
36,./data/blogs_64,64,65,98,0.653061,0.984615,0.785276
21,./data/news1_40,18,18,26,0.692308,1.000000,0.818182
35,./data/blogs_103,118,119,168,0.702381,0.991597,0.822300


In [553]:
results.F1.mean()

0.8888192175356536